In [1]:
import sys
print(sys.executable)
import pandas as pd
print(pd.__version__)

/Users/ocarmeli/Library/Caches/pypoetry/virtualenvs/first-project-fykf2K4j-py3.14/bin/python
2.3.3


In [2]:
import pandas as pd

# ===============================
# Load CSV with date parsing
# ===============================
df = pd.read_csv(
    '/Users/ocarmeli/dev/First_project/data/ads_events.csv',
    parse_dates=['impression_ts', 'click_ts']
)

# Quick look at the first rows
print("=== Head of Data ===")
print(df.head())


=== Head of Data ===
   user_id  ad_id       impression_ts            click_ts  purchase_value
0       36      2 2025-12-01 10:56:00 2025-12-01 11:13:00           100.0
1      165      4 2025-12-01 09:26:00 2025-12-01 10:24:00           200.0
2       55      5 2025-12-01 10:25:00                 NaT             NaN
3       30      4 2025-12-01 09:00:00 2025-12-01 09:27:00            50.0
4       79      3 2025-12-01 10:56:00 2025-12-01 11:01:00             NaN


In [3]:
# ===============================
# General info
# ===============================
print("\n=== DataFrame Info ===")
print(df.info())  # shows dtypes, non-null counts, memory usage

# Summary statistics for numeric columns
print("\n=== Numeric Summary ===")
print(df.describe())



=== DataFrame Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   user_id         200 non-null    int64         
 1   ad_id           200 non-null    int64         
 2   impression_ts   195 non-null    datetime64[ns]
 3   click_ts        64 non-null     datetime64[ns]
 4   purchase_value  46 non-null     float64       
dtypes: datetime64[ns](2), float64(1), int64(2)
memory usage: 7.9 KB
None

=== Numeric Summary ===
          user_id       ad_id                  impression_ts  \
count  200.000000  200.000000                            195   
mean    94.240000  177.350000  2025-12-01 11:44:45.230769152   
min      2.000000    1.000000            2025-12-01 07:06:00   
25%     49.000000    2.000000            2025-12-01 09:13:00   
50%     96.500000    4.000000            2025-12-01 11:45:00   
75%    142.000000    5.00

In [4]:

# ===============================
# Missing values check
# ===============================
print("\n=== Missing Values ===")
print(df.isna().sum())  # how many missing per column



=== Missing Values ===
user_id             0
ad_id               0
impression_ts       5
click_ts          136
purchase_value    154
dtype: int64


In [5]:

# ===============================
# Duplicate check
# ===============================
# Check if same user clicked same ad multiple times (optional)
duplicates = df.duplicated(subset=['user_id', 'ad_id', 'impression_ts'])
print(f"\nNumber of duplicate rows by user/ad/impression: {duplicates.sum()}")



Number of duplicate rows by user/ad/impression: 0


In [6]:

# ===============================
# Check edge cases in timestamps
# ===============================
# Click before impression
click_before_impression = df[(df['click_ts'].notna()) & (df['impression_ts'].notna()) & (df['click_ts'] < df['impression_ts'])]
print(f"\nClicks before impressions: {len(click_before_impression)}")
print(click_before_impression.head())



Clicks before impressions: 7
     user_id  ad_id       impression_ts            click_ts  purchase_value
45       119    999 2025-12-01 12:09:00 2025-12-01 12:04:00            50.0
49       148      2 2025-12-01 13:33:00 2025-12-01 13:23:00             NaN
82       143    999 2025-12-01 15:18:00 2025-12-01 15:15:00           200.0
139      147      3 2025-12-01 15:00:00 2025-12-01 14:56:00             NaN
161      123      1 2025-12-01 08:06:00 2025-12-01 08:02:00             NaN


In [7]:

# ===============================
# Check purchase_value anomalies
# ===============================
# Negative purchases
negative_purchase = df[df['purchase_value'] < 0]
print(f"\nNegative purchases: {len(negative_purchase)}")
print(negative_purchase.head())
# Purchase without click
purchase_without_click = df[(df['purchase_value'].notna()) & (df['click_ts'].isna())]
print(f"\nPurchases without clicks: {len(purchase_without_click)}")
print(purchase_without_click.head())



Negative purchases: 10
     user_id  ad_id       impression_ts            click_ts  purchase_value
25        14      1 2025-12-01 14:13:00 2025-12-01 14:45:00           -10.0
33        49      3 2025-12-01 07:45:00 2025-12-01 08:43:00           -10.0
67        87      1 2025-12-01 14:59:00 2025-12-01 15:17:00           -10.0
81        85      5 2025-12-01 12:21:00 2025-12-01 12:49:00           -10.0
111      164      1 2025-12-01 07:25:00 2025-12-01 07:32:00           -10.0

Purchases without clicks: 0
Empty DataFrame
Columns: [user_id, ad_id, impression_ts, click_ts, purchase_value]
Index: []


In [8]:
import pandas as pd
from src.data_analysis import clean_ads_data, compute_ad_metrics 

# Load CSV
df = pd.read_csv('data/ads_events.csv', parse_dates=['impression_ts', 'click_ts'])

# Define valid ad_ids
valid_ad_ids = [1, 2, 3, 4, 5]

# Clean data
df_cleaned = clean_ads_data(df, valid_ad_ids)

# Compute aggregated metrics
ad_metrics = compute_ad_metrics(df_cleaned)

# Display results
print(ad_metrics.head())


   ad_id  impressions  clicks  purchases     ctr  conversion_rate  \
0      1           32      13         10  0.4062           0.7692   
1      2           29      12          9  0.4138           0.7500   
2      3           30       7          4  0.2333           0.5714   
3      4           24       8          4  0.3333           0.5000   
4      5           42      11         10  0.2619           0.9091   

   avg_purchase_value  
0               30.00  
1               68.33  
2               42.86  
3               40.00  
4               33.64  


In [9]:

# ===============================
# 8. Check ad distribution
# ===============================
ad_counts = df['ad_id'].value_counts()
print("\n=== Ad Distribution ===")
print(ad_counts)



=== Ad Distribution ===
ad_id
5      43
1      35
999    35
2      31
3      31
4      25
Name: count, dtype: int64


In [10]:

# ===============================
# 10. Optional: Check users with multiple ads
# ===============================
users_multiple_ads = df.groupby('user_id')['ad_id'].nunique()
users_multiple_ads = users_multiple_ads[users_multiple_ads > 1]
print(f"\nUsers who saw multiple ads: {len(users_multiple_ads)}")



Users who saw multiple ads: 44
